In [1]:
from random import randint
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 14 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: bool(2), float64(11), int64(1)
memory usage: 3.9+ MB


In [3]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################


df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('data/event_por_hora.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
"""df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
#df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
#df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos = df_unidos.join(df16).fillna(0)"""

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 22 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
timestamp                      38829 non-null int64
min_timestamp                  38829 non-null int64
sum_timestamp                  388

In [4]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 23 columns):
cant_conversions               19414 non-null float64
cant_checkouts                 19414 non-null float64
cant_viewed_product            19414 non-null float64
cant_searched_product          19414 non-null float64
cant_ad_campaign_hit           19414 non-null float64
ad_campaign_hit                19414 non-null bool
cant_lead                      19414 non-null float64
lead                           19414 non-null bool
cant_brand_listing             19414 non-null float64
cant_visited_site              19414 non-null float64
cant_generic_listing           19414 non-null float64
total_sesiones                 19414 non-null int64
promedio_eventos_por_sesion    19414 non-null float64
proprosion                     19414 non-null float64
timestamp                      19414 non-null int64
min_timestamp                  19414 non-null int64
sum_timestamp                  194

In [5]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['cant_conversions', 'cant_checkouts', 'cant_viewed_product', 'cant_searched_product', 'cant_ad_campaign_hit', 'ad_campaign_hit', 'cant_lead', 'lead', 'cant_brand_listing', 'cant_visited_site', 'cant_generic_listing', 'total_sesiones', 'promedio_eventos_por_sesion', 'proprosion', 'timestamp', 'min_timestamp', 'sum_timestamp', 'prom_timestamp', 'ult_view', 'ult_check', 'ult_search', 'ult_conv']


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [7]:
param_grid = { 
    'max_depth' : [2,3,4,5,6,7,8,9,10],
    
} 

In [8]:
from sklearn.model_selection import GridSearchCV 
clf = tree.DecisionTreeClassifier()
CV_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=-1, verbose=10) 
CV_clf.fit(X_train, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=2 .....................................................
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.833754746327, total=   0.1s
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.833069407154, total=   0.1s
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.826460391523, total=   0.1s
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.833230180426, total=   0.1s
[CV] ................ max_depth=2, score=0.820062329142, total=   0.1s
[CV] max_depth=3 .....................................................
[CV] max_depth=3 .....................................................
[CV] ................ max_depth=3, score=0.856201504922, total=   0.1s
[CV] max_depth=3 

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.5s


[CV] ................ max_depth=3, score=0.834945129422, total=   0.1s
[CV] max_depth=3 .....................................................
[CV] ................. max_depth=3, score=0.84762602515, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................ max_depth=3, score=0.837951886277, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................. max_depth=4, score=0.85921221837, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................ max_depth=4, score=0.843264777694, total=   0.1s
[CV] max_depth=4 .....................................................


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s


[CV] ................ max_depth=4, score=0.849897687876, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................ max_depth=4, score=0.837277200656, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=4, score=0.838544559869, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.856784749194, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.844232507016, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.841465083544, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.837554948059, total=   0.1s
[CV] max_depth=6 .....................................................
[CV] .

...

[CV] max_depth=7 .....................................................
[CV] ................ max_depth=7, score=0.804915109176, total=   0.1s
[CV] max_depth=7 .....................................................
[CV] ................ max_depth=7, score=0.823334549783, total=   0.2s
[CV] ................ max_depth=7, score=0.797496992892, total=   0.1s
[CV] max_depth=8 .....................................................
[CV] ................ max_depth=7, score=0.849897211591, total=   0.1s
[CV] max_depth=8 .....................................................
[CV] max_depth=8 .....................................................
[CV] ................ max_depth=8, score=0.812602637958, total=   0.2s
[CV] max_depth=8 .....................................................
[CV] ................ max_depth=8, score=0.801181042498, total=   0.2s
[CV] max_depth=8 .....................................................
[CV] ................. max_depth=8, score=0.80123426218, total=   0.2s
[CV] m

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.0s


[CV] ................ max_depth=9, score=0.792716940802, total=   0.2s
[CV] max_depth=9 .....................................................
[CV] ................. max_depth=9, score=0.80452953801, total=   0.2s
[CV] max_depth=9 .....................................................
[CV] ................ max_depth=9, score=0.764381261459, total=   0.2s
[CV] max_depth=10 ....................................................
[CV] ................ max_depth=9, score=0.754137780208, total=   0.2s
[CV] max_depth=10 ....................................................
[CV] ................ max_depth=9, score=0.757959540733, total=   0.2s
[CV] max_depth=10 ....................................................
[CV] ............... max_depth=10, score=0.763498249181, total=   0.2s
[CV] max_depth=10 ....................................................
[CV] ............... max_depth=10, score=0.781350737256, total=   0.2s
[CV] max_depth=10 ....................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:    3.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    4.1s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [9]:
CV_clf.best_params_ #aca te dice el mejor hiperparametro

{'max_depth': 5}

In [10]:
clf = tree.DecisionTreeClassifier(max_depth= 5)
clf.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
clf.score(X_test,Y_test)

0.94540303888745814

In [12]:
Y_pred = clf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.519696407294


In [13]:
Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      3681
          1       0.32      0.04      0.08       202

avg / total       0.92      0.95      0.93      3883



In [14]:
print(confusion_matrix(Y_truths, Y_pred))

[[3662   19]
 [ 193    9]]


In [15]:
feature_importances = pd.DataFrame(clf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [16]:
feature_importances.head(60)

,importance
ult_check,0.589330
ult_conv,0.122028
sum_timestamp,0.073485
cant_viewed_product,0.057967
promedio_eventos_por_sesion,0.039189
cant_generic_listing,0.019117
cant_conversions,0.015760
cant_brand_listing,0.015181
ult_search,0.013677
cant_checkouts,0.013083


In [17]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [18]:
kaggle_pred = clf.predict_proba(df_events)
kaggle_pred
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [19]:
df_submit.label.mean()

0.050838721787373704

In [20]:
df_submit.to_csv('submit/submit_clf_clase.csv')

In [21]:
len(df_submit['label'])


19415